In [17]:
from lis import evaluate

In [18]:
p1 = "(begin (define r 10) (* pi (* r r)))"
evaluate(p1)

314.1592653589793

In [19]:
p2 = ("(car (list 1 2 3 4))")
evaluate(p2)

1

In [29]:
p3 = """
(begin 
  (define fact (lambda (n) (if (<= n 1) 1 (* n (fact (- n 1))))))
  (fact 100)
)
"""
%timeit evaluate(p3)
evaluate(p3)

1.24 ms ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [27]:
fact = lambda n: 1 if n<=1 else n * fact(n-1)
%timeit fact(100)
fact(100)


13.5 µs ± 37.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [21]:
p4 = """
(begin 
  (define twice (lambda (x) (* 2 x)))
  (twice 10)
)
"""

evaluate(p4)

20

In [22]:
p5 = """
(begin
  (define fib (lambda (n) (if (< n 2) 1 (+ (fib (- n 1)) (fib (- n 2))))))
  (fib 10)
)
"""
%timeit evaluate(p5)
evaluate(p5)

1.83 ms ± 7.82 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


89

In [23]:
fib = lambda n: 1 if n < 2 else fib(n - 1) + fib (n - 2)
%timeit fib(10)
fib(10)

13.9 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


89

In [24]:
p6 = """
(begin
  (define move (lambda (n from to spare)
    (if 
      (= 0 n) "Done_Moving"
      (begin
        (move (- n 1) from spare to)
        (print (list "Move" n "from" from "to" to))
        (move (- n 1) spare to from)
      )
    ))
)
  (move 3 "One" "Three" "Two")
)
"""

evaluate(p6)

['"Move"', 1, '"from"', '"One"', '"to"', '"Three"']
['"Move"', 2, '"from"', '"One"', '"to"', '"Two"']
['"Move"', 1, '"from"', '"Three"', '"to"', '"Two"']
['"Move"', 3, '"from"', '"One"', '"to"', '"Three"']
['"Move"', 1, '"from"', '"Two"', '"to"', '"One"']
['"Move"', 2, '"from"', '"Two"', '"to"', '"Three"']
['"Move"', 1, '"from"', '"One"', '"to"', '"Three"']


'"Done_Moving"'

In [25]:
p7 ="""
(begin
  (define threshold 1e-6)
    
  (define newton (lambda (f guess)
    (fixed-point (lambda (x) (- x (/ (f x) ((derive f threshold) x)))) guess)))

  (define derive (lambda (f dx)
    (lambda (x) (/ (- (f (+ x dx)) (f x)) dx))))

  (define close-enough? (lambda (x y)
    (< (abs (- x y)) threshold)))
    
  (define fixed-point (lambda (f n)
    (if (close-enough? n (f n))
        n
        (fixed-point f (f n)))))

  (define jazr (lambda (y)
    (newton (lambda (x) (- y (* x x)))  1.0)))

  (jazr 2)
)
"""

evaluate(p7)

1.4142135623754424